# 4th attempt - CNN

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
SIZE = 10
AMOUNT_BOARDS = 1000

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [5]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 41366
reverse df sort: 41366


In [6]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  33506
len x test:  4137
len y train:  33506
len y test:  4137


In [7]:
X_train.shape

(33506, 100)

In [8]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [9]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 100)
(33506,)


In [10]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [11]:
print(X_train_array.shape)
print(y_train_array.shape)

(33506, 10, 10, 1)
(33506, 1)


In [12]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [13]:
import tensorflow as tf

input_shape = (SIZE, SIZE, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(), 

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 10, 10, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 5, 5, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,329 (138.00 KB)

 Trainable params: 35,329 (138.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
history = model.fit(X_train_array, y_train_array, 
    validation_data=(X_val_array, y_val_array),
    epochs=10,
    batch_size=128)


Epoch 1/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.7838 - loss: 0.4808 - val_accuracy: 0.8238 - val_loss: 0.3801
Epoch 2/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.8217 - loss: 0.3776 - val_accuracy: 0.8276 - val_loss: 0.3701
Epoch 3/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8265 - loss: 0.3661 - val_accuracy: 0.8335 - val_loss: 0.3607
Epoch 4/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8393 - loss: 0.3517 - val_accuracy: 0.8380 - val_loss: 0.3513
Epoch 5/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8424 - loss: 0.3428 - val_accuracy: 0.8439 - val_loss: 0.3489
Epoch 6/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8468 - loss: 0.3345 - val_accuracy: 0.8442 - val_loss: 0.3474
Epoch 7/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8513 - loss: 0.3279 - val_accuracy: 0.8396 - val_loss: 0.3558
Epoch 8/10
262/262 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.8495 - loss: 0.3257 - val_acc

In [16]:
test_loss, test_acc = model.evaluate(X_test_array, y_test_array)
print(f"Test accuracy: {test_acc:.3f}")

130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8288 - loss: 0.3765
Test accuracy: 0.833
